# 🏢 Sistem Pencarian Perusahaan KarirHub™

**Notebook Crawling Data Perusahaan dari Platform KarirHub**

---

## 📋 Deskripsi Proyek

Notebook ini berisi implementasi sistem web scraping untuk mengekstrak informasi perusahaan dari platform KarirHub. Sistem ini dirancang untuk mengumpulkan data perusahaan secara otomatis dengan antarmuka yang user-friendly.

## 🎯 Tujuan

- Mengumpulkan data perusahaan dari KarirHub secara otomatis
- Menyediakan antarmuka interaktif untuk pencarian perusahaan
- Mengekstrak informasi detail seperti nama perusahaan, lokasi, dan deskripsi
- Menyimpan hasil crawling dalam format yang terstruktur

## 🔧 Teknologi yang Digunakan

- **Python**: Bahasa pemrograman utama
- **Selenium**: Web scraping dan otomasi browser
- **IPython Widgets**: Antarmuka interaktif dalam notebook
- **Pandas**: Manipulasi dan analisis data

## 📝 Struktur Notebook

1. **Import Library**: Mengimpor semua pustaka yang diperlukan
2. **Konfigurasi Selenium**: Setup driver dan konfigurasi browser
3. **Implementasi Widget**: Antarmuka interaktif untuk pencarian
4. **Eksekusi Pencarian**: Menjalankan proses crawling data

---

*Dikembangkan untuk keperluan penelitian dan analisis data perusahaan*

**© 2025 - Ferdian Bangkit Wijaya - UNTIRTA**  
**📧 Contact**: ferdian.bangkit@untirta.ac.id

## 📦 Import Library dan Dependensi

Cell berikut ini mengimpor semua pustaka (library) yang diperlukan untuk menjalankan sistem web scraping KarirHub™. Setiap library memiliki fungsi spesifik:

- **`selenium`**: Library utama untuk web scraping dan otomasi browser
- **`webdriver_manager`**: Mengelola driver browser secara otomatis
- **`ipywidgets`**: Membuat antarmuka interaktif dalam Jupyter Notebook
- **`IPython.display`**: Menampilkan widget dan output dalam notebook
- **`time`**: Mengatur delay dan timing dalam proses scraping

Pastikan semua library telah terinstall sebelum menjalankan cell ini.

In [1]:
# Cell 1: Import Library dan Install Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import untuk widget interaktif
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Import untuk Selenium
try:
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from webdriver_manager.chrome import ChromeDriverManager
    print("✅ Selenium libraries berhasil di-import!")
except ImportError:
    print("❌ Selenium belum terinstall. Menginstall selenium dan webdriver-manager...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "selenium", "webdriver-manager"])
    print("✅ Selenium berhasil diinstall! Silakan restart kernel dan jalankan ulang cell ini.")

print("✅ Semua library berhasil di-import!")
print("📦 Library yang digunakan:")
print("- requests: untuk HTTP requests")
print("- beautifulsoup4: untuk parsing HTML")
print("- pandas: untuk manipulasi data")
print("- ipywidgets: untuk widget interaktif")
print("- urllib: untuk encoding URL")
print("- selenium: untuk browser automation")
print("- webdriver-manager: untuk Chrome driver management")

✅ Selenium libraries berhasil di-import!
✅ Semua library berhasil di-import!
📦 Library yang digunakan:
- requests: untuk HTTP requests
- beautifulsoup4: untuk parsing HTML
- pandas: untuk manipulasi data
- ipywidgets: untuk widget interaktif
- urllib: untuk encoding URL
- selenium: untuk browser automation
- webdriver-manager: untuk Chrome driver management


## ⚙️ Konfigurasi Selenium WebDriver

Cell ini berisi konfigurasi dan setup untuk Selenium WebDriver yang akan digunakan untuk mengakses website KarirHub™. Proses yang dilakukan meliputi:

### 🔧 Fungsi Utama:
- **Setup Chrome Options**: Mengatur parameter browser Chrome untuk optimasi scraping
- **WebDriver Manager**: Otomatis mendownload dan mengelola ChromeDriver
- **Browser Initialization**: Menginisialisasi instance browser Chrome

### 📋 Konfigurasi Browser:
- Mode headless atau dengan GUI (dapat disesuaikan)
- User-agent untuk menghindari deteksi bot
- Window size dan pengaturan lainnya untuk stabilitas scraping

Jalankan cell ini untuk mempersiapkan browser yang akan digunakan dalam proses crawling data.

In [2]:
# Cell 2: Fungsi Crawling KarirHub dengan Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time

class KarirHubCrawler:
    def __init__(self):
        self.base_url = "https://karirhub.kemnaker.go.id"
        self.driver = None
        self.stop_crawling = False  # Flag untuk menghentikan crawling
        self.setup_driver()
    
    def setup_driver(self):
        """Setup Chrome driver dengan options yang optimal"""
        print("🔧 Menyiapkan Chrome driver...")
        
        # Chrome options - Background mode
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # Run in background
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--disable-logging')
        chrome_options.add_argument('--disable-extensions')
        chrome_options.add_argument('--disable-web-security')
        chrome_options.add_argument('--window-size=1920,1080')
        chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        
        # Install and setup ChromeDriver
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
        print("✅ Chrome driver siap! (Background mode)")
    
    def stop(self):
        """Method untuk menghentikan crawling"""
        self.stop_crawling = True
        print("🛑 Stop signal diterima - akan berhenti setelah halaman saat ini selesai")
    
    def search_companies(self, keyword):
        """
        Mencari perusahaan berdasarkan keyword menggunakan Selenium dengan pagination click
        """
        companies = []
        self.stop_crawling = False  # Reset flag
        
        try:
            # Encode keyword untuk URL
            encoded_keyword = urllib.parse.quote(keyword)
            search_url = f"{self.base_url}/perusahaan?filters=keyword:{encoded_keyword}%23{encoded_keyword}"
            
            print(f"🔍 Mencari perusahaan dengan keyword: '{keyword}' (Auto-crawl dengan click pagination)")
            print(f"🌐 URL: {search_url}")
            print(f"📄 Crawling dengan mengklik pagination sampai next button disabled...")
            
            # Load halaman pertama
            print(f"📄 Memuat halaman pertama...")
            self.driver.get(search_url)
            
            # Wait for page to load
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            time.sleep(2)
            
            page = 1
            while not self.stop_crawling:
                try:
                    print(f"📄 Memproses halaman {page}...")
                    
                    # Wait for content to load
                    WebDriverWait(self.driver, 8).until(
                        EC.presence_of_element_located((By.TAG_NAME, "sisnaker-element-karirhub-company-card-web"))
                    )
                    time.sleep(1)
                    
                    # Cari semua company cards di halaman ini
                    company_cards = []
                    try:
                        company_cards = self.driver.find_elements(By.TAG_NAME, "sisnaker-element-karirhub-company-card-web")
                        print(f"   🎯 Ditemukan {len(company_cards)} company cards")
                    except:
                        print("   ❌ Tidak ada company cards ditemukan")
                    
                    # Jika tidak ada company cards, stop
                    if not company_cards or len(company_cards) == 0:
                        print(f"🛑 Halaman {page} kosong - SELESAI crawling")
                        break
                    
                    # Ekstrak data dari company cards dengan anti-stale mechanism
                    page_companies = []
                    total_cards = len(company_cards)
                    
                    for i in range(total_cards):
                        if self.stop_crawling:
                            print("🛑 Crawling dihentikan oleh user")
                            break
                        
                        print(f"   📋 Memproses card {i+1}/{total_cards}")
                        
                        try:
                            # Tambah waktu tunggu untuk memastikan DOM sudah dimuat
                            time.sleep(1)
                            
                            # RE-FIND company cards setiap iterasi untuk menghindari stale element
                            max_card_retries = 5  # Increased retries
                            fresh_company_cards = []
                            
                            for card_retry in range(max_card_retries):
                                try:
                                    # Try different wait times based on retry attempts
                                    wait_time = 1 + card_retry * 1.5
                                    time.sleep(wait_time)
                                    
                                    fresh_company_cards = self.driver.find_elements(By.TAG_NAME, "sisnaker-element-karirhub-company-card-web")
                                    if len(fresh_company_cards) > 0:
                                        print(f"      ✅ Berhasil menemukan {len(fresh_company_cards)} cards pada retry {card_retry+1}")
                                        break
                                    else:
                                        print(f"      ⚠️ Tidak ada cards ditemukan pada retry {card_retry+1}, mencoba lagi...")
                                        
                                        # Try to refresh page if no cards found after 2 retries
                                        if card_retry >= 2:
                                            print(f"      🔄 Refresh halaman untuk mencoba lagi...")
                                            self.driver.refresh()
                                            WebDriverWait(self.driver, 10).until(
                                                EC.presence_of_element_located((By.TAG_NAME, "body"))
                                            )
                                            time.sleep(3)  # Give extra time after refresh
                                except Exception:
                                    print(f"      ⚠️ Kesalahan saat mencari cards pada percobaan {card_retry+1}")
                                    # Try to scroll to top if there's an error
                                    if card_retry >= 1:
                                        try:
                                            self.driver.execute_script("window.scrollTo(0, 0);")
                                            time.sleep(1)
                                        except:
                                            pass
                            
                            if not fresh_company_cards:
                                print(f"      ❌ Gagal menemukan cards setelah {max_card_retries} retries")
                                # Check if we need to refresh the whole page
                                try:
                                    print(f"      🔄 Refresh halaman setelah gagal menemukan cards...")
                                    self.driver.refresh()
                                    time.sleep(3)
                                except:
                                    pass
                                continue
                            
                            if i >= len(fresh_company_cards):
                                print(f"      ⚠️ Card index {i+1} tidak tersedia dalam fresh cards (hanya {len(fresh_company_cards)} cards)")
                                # Try alternative strategy - find all cards again after scrolling
                                try:
                                    print(f"      🔍 Mencoba strategi alternatif...")
                                    self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
                                    time.sleep(2)
                                    fresh_company_cards = self.driver.find_elements(By.TAG_NAME, "sisnaker-element-karirhub-company-card-web")
                                    
                                    if i < len(fresh_company_cards):
                                        print(f"      ✅ Berhasil menemukan card dengan strategi alternatif")
                                    else:
                                        continue
                                except:
                                    continue
                            
                            # Ambil card yang fresh
                            fresh_card = fresh_company_cards[i]
                            
                            # Scroll element into view before interacting
                            scroll_success = False
                            for scroll_attempt in range(3):
                                try:
                                    # Try different scroll strategies
                                    if scroll_attempt == 0:
                                        self.driver.execute_script(
                                            "arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", 
                                            fresh_card
                                        )
                                    elif scroll_attempt == 1:
                                        # Try scrolling with a different approach
                                        rect = self.driver.execute_script(
                                            "return arguments[0].getBoundingClientRect();", 
                                            fresh_card
                                        )
                                        if rect and 'top' in rect:
                                            self.driver.execute_script(f"window.scrollBy(0, {rect['top'] - 100});")
                                    else:
                                        # Last resort - try to click via JavaScript directly
                                        self.driver.execute_script("arguments[0].scrollIntoView(true);", fresh_card)
                                    
                                    time.sleep(1 + scroll_attempt)  # Wait longer with each attempt
                                    scroll_success = True
                                    break
                                except Exception:
                                    print(f"      ℹ️ Percobaan scroll ke-{scroll_attempt+1} sedang diproses...")
                            if not scroll_success:
                                print(f"      ⚠️ Warning: Tidak bisa scroll ke card, tapi tetap mencoba proses")
                            
                            # Set up max retries for processing a specific card
                            max_process_retries = 3
                            processing_success = False
                            company_data = None
                            
                            for process_retry in range(max_process_retries):
                                try:
                                    print(f"      🔄 Processing attempt {process_retry+1}/{max_process_retries} for card {i+1}")
                                    company_data = self.extract_company_info(fresh_card)
                                    
                                    if company_data and company_data.get('name'):
                                        page_companies.append(company_data)
                                        print(f"   ✅ Card {i+1} berhasil diproses: {company_data['name']}")
                                        processing_success = True
                                        break
                                    else:
                                        print(f"   ⚠️ Card {i+1} attempt {process_retry+1} gagal, data kosong")
                                        # Wait before retrying
                                        time.sleep(2 + process_retry)
                                except Exception as process_err:
                                    print(f"   ⚠️ Kendala pada kartu {i+1}, percobaan {process_retry+1}")
                                    
                                    # Different recovery strategies based on error type and retry count
                                    error_msg = str(process_err).lower()
                                    if "stale element" in error_msg:
                                        print(f"      🔄 Elemen tidak segar, mencari kartu ulang...")
                                        try:
                                            # Try to find this card again
                                            fresh_company_cards = self.driver.find_elements(By.TAG_NAME, "sisnaker-element-karirhub-company-card-web")
                                            if i < len(fresh_company_cards):
                                                fresh_card = fresh_company_cards[i]
                                                time.sleep(1)
                                        except:
                                            pass
                                    elif "no such element" in error_msg:
                                        print(f"      🔍 Elemen tidak ditemukan, menyesuaikan strategi...")
                                        # Try different approach on each retry
                                        if process_retry == 0:
                                            # First retry - just wait a bit longer
                                            time.sleep(3)
                                        elif process_retry == 1:
                                            # Second retry - try scrolling to make sure element is visible
                                            try:
                                                self.driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", fresh_card)
                                                time.sleep(2)
                                            except:
                                                pass
                                        else:
                                            # Last retry - try refreshing the page
                                            print(f"      🔄 Percobaan terakhir: Refresh halaman...")
                                            try:
                                                current_url = self.driver.current_url
                                                self.driver.get(current_url)
                                                time.sleep(3)
                                                
                                                # Re-find cards after refresh
                                                fresh_company_cards = self.driver.find_elements(By.TAG_NAME, "sisnaker-element-karirhub-company-card-web")
                                                if i < len(fresh_company_cards):
                                                    fresh_card = fresh_company_cards[i]
                                                else:
                                                    print(f"      ℹ️ Kartu {i+1} tidak tersedia setelah refresh")
                                                    break
                                            except Exception:
                                                print(f"      ℹ️ Proses refresh tidak berhasil")
                                                break
                                    else:
                                        print(f"      ⚠️ Unexpected error, mencoba recovery strategy...")
                                        # Generic recovery - wait and continue
                                        time.sleep(2 + process_retry)
                            
                            if not processing_success:
                                print(f"   ❌ Failed to process card {i+1} after {max_process_retries} attempts")
                                # Create minimal record if we've failed to extract but know there should be a company
                                if i+1 <= len(fresh_company_cards):
                                    try:
                                        fallback_company = {
                                            'name': f"Company #{i+1} on page {page}",
                                            'location': 'Unknown',
                                            'industry': 'Unknown',
                                            'jobs_available': 0,
                                            'company_url': '',
                                            'description': f'Failed to extract details for company #{i+1} on page {page} with keyword {keyword}'
                                        }
                                        # Try to at least get the name if possible
                                        try:
                                            name_elem = fresh_card.find_element(By.CSS_SELECTOR, "div.font-bold")
                                            if name_elem and name_elem.text.strip():
                                                fallback_company['name'] = name_elem.text.strip()
                                        except Exception:
                                            pass
                                        
                                        page_companies.append(fallback_company)
                                        print(f"      ℹ️ Created fallback record for {fallback_company['name']}")
                                    except Exception:
                                        pass
                        except Exception as e:
                            print(f"   ❌ Critical error pada card {i+1}: {str(e)}")
                            # Check if we need to refresh the whole page after too many errors
                            if i > 0 and i % 3 == 0:
                                try:
                                    print(f"      🔄 Critical error recovery: Refresh halaman...")
                                    self.driver.refresh()
                                    time.sleep(3)
                                except:
                                    pass
                    
                    companies.extend(page_companies)
                    print(f"✅ Halaman {page}: {len(page_companies)} perusahaan berhasil diekstrak")
                    
                    if self.stop_crawling:
                        print("🛑 Crawling dihentikan oleh user")
                        break
                    
                    # Cek apakah ada next button yang bisa diklik
                    next_available = self.check_and_click_next_page()
                    if not next_available:
                        print(f"🏁 Next button disabled - SELESAI crawling")
                        break
                    
                    page += 1
                        
                except Exception as e:
                    print(f"❌ Error pada halaman {page}: {str(e)}")
                    break
            
            if self.stop_crawling:
                print(f"🛑 Crawling dihentikan! Total perusahaan ditemukan: {len(companies)} dari {page} halaman")
            else:
                print(f"🎉 Crawling selesai! Total perusahaan ditemukan: {len(companies)} dari {page} halaman")
            return companies
            
        except Exception as e:
            print(f"❌ Error dalam pencarian: {str(e)}")
            return []
    
    def check_and_click_next_page(self):
        """
        Cek apakah next button tersedia dan klik jika bisa
        Return True jika berhasil klik, False jika next button disabled
        """
        try:
            # Cari pagination container
            pagination = self.driver.find_element(By.CSS_SELECTOR, "div.space-x-2.pagination.flex-1")
            
            # Cari next button (chevron-right)
            next_buttons = pagination.find_elements(By.CSS_SELECTOR, "ion-button i-feather[name='chevron-right']")
            
            for next_button in next_buttons:
                # Ambil parent button element
                parent_button = next_button.find_element(By.XPATH, "./..")
                
                # Cek apakah button disabled
                is_disabled = parent_button.get_attribute("disabled")
                has_disabled_class = "disabled" in parent_button.get_attribute("class")
                
                if is_disabled or has_disabled_class:
                    print("   🔄 Next button disabled - ini adalah halaman terakhir")
                    return False
                else:
                    # Next button tersedia, klik
                    print("   ➡️ Mengklik next button...")
                    self.driver.execute_script("arguments[0].click();", parent_button)
                    time.sleep(2)  # Wait for page transition
                    return True
            
            print("   ❌ Next button tidak ditemukan")
            return False
            
        except Exception as e:
            print(f"   ⚠️ Error checking next button: {str(e)}")
            return False
    
    def extract_company_info(self, card_element):
        """
        Ekstrak informasi perusahaan dengan mengklik detail dan kembali ke hasil pencarian
        Termasuk mekanisme anti-elemen basi (anti-stale)
        Ditingkatkan dengan penanganan kesalahan dan pemulihan yang lebih baik
        """
        try:
            company_data = {
                'name': '',
                'location': '',
                'industry': '',
                'website': '',
                'description': '',
                'logo_url': '',
                'jobs_available': 0,
                'company_url': '',
                'phone': '',
                'email': '',
                'registered_since': ''
            }
            
            # Simpan URL halaman saat ini (hasil pencarian)
            current_url = self.driver.current_url
            
            # Cari div utama company card dengan stale element protection
            main_div = None
            max_retries = 5  # Increased retries
            
            for retry in range(max_retries):
                try:
                    # Cek apakah elemen sudah basi (stale)
                    try:
                        tag_name = card_element.tag_name  # Ini akan melempar exception jika stale
                    except Exception as stale_error:
                        print(f"      ⚠️ Elemen basi terdeteksi pada percobaan {retry+1}, mencari ulang...")
                        
                        # Cari ulang elemen card dengan mendapatkan semua card lagi
                        wait_time = 2 + retry  # Tingkatkan waktu tunggu pada setiap percobaan
                        time.sleep(wait_time)  # Tambahkan waktu tunggu ekstra sebelum percobaan ulang
                        
                        try:
                            fresh_cards = self.driver.find_elements(By.TAG_NAME, "sisnaker-element-karirhub-company-card-web")
                            if len(fresh_cards) > 0:
                                # Coba temukan card yang mirip dengan yang sedang kita proses
                                if hasattr(card_element, 'text') and card_element.text:
                                    # Cari konten teks yang mirip
                                    best_match = fresh_cards[0]
                                    for fresh_card in fresh_cards:
                                        if fresh_card.text and card_element.text in fresh_card.text:
                                            best_match = fresh_card
                                            break
                                    card_element = best_match
                                else:
                                    card_element = fresh_cards[0]  # Gunakan yang pertama tersedia sebagai cadangan
                                    
                                print(f"      ✅ Elemen card baru ditemukan")
                            else:
                                print(f"      ❌ Tidak dapat menemukan elemen card baru")
                                return None
                        except Exception:
                            print(f"      ℹ️ Mencari ulang elemen pada percobaan {retry+1}...")
                            if retry == max_retries - 1:
                                return None
                            continue
                    
                    # Coba scroll elemen agar terlihat dulu
                    try:
                        self.driver.execute_script("arguments[0].scrollIntoView({block: 'center', behavior: 'smooth'});", card_element)
                        time.sleep(1)  # Tunggu untuk scroll selesai
                    except Exception:
                        print(f"      ℹ️ Perlu penyesuaian scroll")
                    
                    # Sekarang coba temukan div utama dalam card
                    try:
                        if card_element.tag_name == 'sisnaker-element-karirhub-company-card-web':
                            # Coba beberapa strategi selector
                            try:
                                main_div = card_element.find_element(By.CSS_SELECTOR, "div.p-4.border.border-grey-200.rounded-xl.cursor-pointer.h-full")
                            except:
                                # Selector alternatif jika yang pertama gagal
                                try:
                                    main_div = card_element.find_element(By.CSS_SELECTOR, "div.p-4")
                                except:
                                    # Upaya terakhir: cari div yang bisa diklik
                                    main_div = card_element.find_element(By.CSS_SELECTOR, "div[class*='cursor-pointer']")
                        else:
                            main_div = card_element
                    except Exception as div_err:
                        print(f"      ⚠️ Kesalahan saat mencari div pada percobaan {retry+1}: {str(div_err)}")
                        # Coba dengan JavaScript jika metode normal gagal
                        if retry >= 2:
                            try:
                                main_div = self.driver.execute_script("return arguments[0].querySelector('div.p-4') || arguments[0].querySelector('div[class*=\"cursor-pointer\"]') || arguments[0];", card_element)
                                if main_div:
                                    print("      ✅ Elemen ditemukan menggunakan JavaScript")
                            except:
                                pass
                    
                    if main_div:
                        break  # Success, exit retry loop
                    
                except Exception as e:
                    print(f"      ⚠️ Kesalahan saat mencari elemen pada percobaan {retry+1}: {str(e)}")
                    time.sleep(1 + retry)  # Tingkatkan waktu tunggu pada setiap percobaan
                    if retry == max_retries - 1:
                        main_div = card_element  # Upaya terakhir
            
            if not main_div:
                return None
            
            # 1. Ekstrak informasi basic dari card (sebelum klik detail)
            try:
                name_elem = main_div.find_element(By.CSS_SELECTOR, "div.font-bold.text-base.leading-5\\.5.mb-1")
                company_data['name'] = name_elem.text.strip()
            except:
                pass
            
            try:
                industry_elem = main_div.find_element(By.CSS_SELECTOR, "div.text-grey-700.font-normal.text-sm.leading-5.mb-1")
                industry_text = industry_elem.text.strip()
                if industry_text and industry_text != 'Not specified':
                    company_data['industry'] = industry_text
            except:
                pass
            
            try:
                location_elem = main_div.find_element(By.CSS_SELECTOR, "div.text-grey-500.leading-3\\.5.text-xs.mb-4")
                location_text = location_elem.text.strip()
                location_text = location_text.replace(', Indonesia', '').strip()
                import re
                location_text = re.sub(r'\\s+', ' ', location_text).strip()
                if location_text:
                    company_data['location'] = location_text
            except:
                pass
            
            try:
                rating_elem = main_div.find_element(By.CSS_SELECTOR, "span.text-sm.mr-1")
                rating_text = rating_elem.text.strip()
                if rating_text.isdigit():
                    company_data['jobs_available'] = int(rating_text)
            except:
                pass
            
            # Coba ekstrak URL perusahaan sebelum mengklik (untuk berjaga-jaga jika tidak bisa kembali ke halaman hasil)
            try:
                # Percobaan pertama: Ekstrak company_url dari card sebelum navigasi
                company_url_element = None
                
                # Coba beberapa selector untuk menemukan URL dalam card
                for url_selector in ["a[href*='/company/']", "a[href*='/perusahaan/']", "a"]:
                    try:
                        company_url_element = main_div.find_element(By.CSS_SELECTOR, url_selector)
                        if company_url_element:
                            href = company_url_element.get_attribute('href')
                            if href and ('/company/' in href or '/perusahaan/' in href):
                                company_data['company_url'] = href
                                print(f"      ✅ Berhasil ekstrak URL perusahaan sebelum navigasi: {company_data['company_url']}")
                                break
                    except Exception:
                        continue
            except Exception:
                print(f"      ℹ️ Mencoba cara lain untuk ekstrak URL perusahaan")
            
            # 2. Klik company card untuk masuk ke halaman detail
            try:
                print(f"      🔗 Mengklik detail untuk: {company_data['name']}")
                
                # Coba beberapa strategi klik
                click_success = False
                click_methods = [
                    # Metode 1: Klik JavaScript
                    lambda: self.driver.execute_script("arguments[0].click();", main_div),
                    # Metode 2: ActionChains gerak dan klik
                    lambda: ActionChains(self.driver).move_to_element(main_div).click().perform(),
                    # Metode 3: Klik langsung
                    lambda: main_div.click(),
                    # Metode 4: Coba temukan dan klik tag anchor di dalamnya
                    lambda: main_div.find_element(By.TAG_NAME, "a").click()
                ]
                
                for i, click_method in enumerate(click_methods):
                    try:
                        click_method()
                        click_success = True
                        print(f"      ✅ Klik berhasil dengan metode {i+1}")
                        break
                    except Exception:
                        print(f"      ℹ️ Mencoba metode klik alternatif ({i+1})...")
                        if i < len(click_methods) - 1:
                            print(f"      🔄 Menggunakan strategi klik berikutnya...")
                            time.sleep(1)  # Tunggu sebentar sebelum metode berikutnya
                
                if not click_success:
                    print(f"      ❌ Semua metode klik gagal, mencoba melanjutkan...")
                
                # Tunggu halaman detail dimuat dengan beberapa kemungkinan selector
                detail_page_selectors = [
                    "div.container.sm\\:px-28",
                    "div.sisnaker-container",
                    "div.text-xs.sm\\:text-sm.text-grey-400"  # Elemen tanggal pendaftaran
                ]
                
                detail_loaded = False
                for selector in detail_page_selectors:
                    try:
                        WebDriverWait(self.driver, 8).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                        )
                        detail_loaded = True
                        print(f"      ✅ Halaman detail dimuat (terdeteksi dengan selector: {selector})")
                        break
                    except:
                        pass
                
                if detail_loaded:
                    time.sleep(2)  # Berikan waktu ekstra agar semua elemen ter-render
                else:
                    print(f"      ⚠️ Halaman detail mungkin belum dimuat dengan benar")
                    # Periksa apakah URL telah berubah untuk memastikan kita berada di halaman detail
                    if '/company/' in self.driver.current_url or '/perusahaan/' in self.driver.current_url:
                        print(f"      ✅ URL mengkonfirmasi kita berada di halaman detail: {self.driver.current_url}")
                    else:
                        print(f"      ❌ URL menunjukkan kita tidak berada di halaman detail: {self.driver.current_url}")
                        raise Exception("Gagal bernavigasi ke halaman detail")
                
                # Simpan URL detail perusahaan SEBELUM kembali ke hasil pencarian
                detail_url = self.driver.current_url
                if '/perusahaan/' in detail_url or '/company/' in detail_url:
                    company_data['company_url'] = detail_url
                    print(f"      🌐 URL perusahaan disimpan: {detail_url}")
                
                # 3. Ekstrak informasi detail dari halaman detail
                company_data = self.extract_company_detail_info(company_data)
                
                # 4. Kembali ke halaman hasil pencarian
                print(f"      ⬅️ Kembali ke hasil pencarian")
                max_return_retries = 3
                return_success = False
                
                for return_retry in range(max_return_retries):
                    try:
                        self.driver.get(current_url)
                        
                        # Tunggu halaman hasil dimuat kembali dengan verifikasi
                        WebDriverWait(self.driver, 8).until(
                            EC.presence_of_element_located((By.TAG_NAME, "sisnaker-element-karirhub-company-card-web"))
                        )
                        time.sleep(1 + return_retry)  # Tambahkan waktu tunggu yang meningkat dengan setiap percobaan
                        
                        # Verifikasi cards dimuat dengan benar
                        verification_cards = self.driver.find_elements(By.TAG_NAME, "sisnaker-element-karirhub-company-card-web")
                        if len(verification_cards) > 0:
                            print(f"      ✅ Berhasil kembali ke hasil pencarian dengan {len(verification_cards)} cards")
                            return_success = True
                            break
                        else:
                            print(f"      ⚠️ Cards belum dimuat pada percobaan kembali ke-{return_retry+1}")
                            time.sleep(2)  # Waktu tunggu tambahan antara pemeriksaan verifikasi
                    except Exception:
                        print(f"      ℹ️ Navigasi kembali percobaan ke-{return_retry+1}/{max_return_retries}...")
                        time.sleep(2 + return_retry)  # Waktu tunggu bertambah sebelum percobaan berikutnya
                
                if not return_success:
                    print(f"      ⚠️ Gagal kembali ke hasil pencarian setelah {max_return_retries} percobaan, mencoba percobaan akhir...")
                    try:
                        # Percobaan terakhir dengan pendekatan yang lebih kuat
                        self.driver.get(current_url)
                        time.sleep(5)  # Waktu tunggu lebih lama sebagai upaya terakhir
                    except Exception:
                        print(f"      ℹ️ Perlu strategi alternatif navigasi")
                
            except Exception as e:
                print(f"      ⚠️ Kesalahan saat mengakses detail: {str(e)}")
                
                # Jika kita memiliki URL perusahaan tetapi tidak bisa mendapatkan detail lain, setidaknya kita simpan itu
                if not company_data.get('company_url') and ('/company/' in self.driver.current_url or '/perusahaan/' in self.driver.current_url):
                    company_data['company_url'] = self.driver.current_url
                    print(f"      ✅ Menyimpan URL saat ini sebagai cadangan: {company_data['company_url']}")
                
                # Tetap coba kembali ke halaman hasil jika error
                try:
                    self.driver.get(current_url)
                    WebDriverWait(self.driver, 5).until(
                        EC.presence_of_element_located((By.TAG_NAME, "body"))
                    )
                    time.sleep(3)
                except Exception as nav_error:
                    print(f"      ❌ Kesalahan saat kembali ke halaman pencarian: {str(nav_error)}")
                    # Coba sekali lagi dengan waktu tunggu lebih lama
                    try:
                        self.driver.get(current_url)
                        time.sleep(5)  # Paksa menunggu lebih lama
                    except:
                        print("      ⚠️ Gagal kembali ke halaman pencarian setelah beberapa percobaan")
            
            # 5. URL perusahaan sudah disimpan sebelum kembali ke halaman hasil pencarian
            # Cek jika URL masih kosong (untuk backward compatibility)
            if company_data['name'] and not company_data['company_url']:
                print(f"      ⚠️ URL perusahaan tidak tersimpan, mencoba generate fallback URL")
                fallback_url = f"https://karirhub.kemnaker.go.id/perusahaan/{company_data['name'].lower().replace(' ', '-')}"
                company_data['company_url'] = fallback_url
            
            # 6. Generate deskripsi
            if company_data['industry'] and not company_data['description']:
                company_data['description'] = f"Perusahaan bergerak di bidang {company_data['industry']}"
            
            # Return data jika ada nama
            if company_data['name']:
                return company_data
            else:
                return None
            
        except Exception:
            print(f"      ℹ️ Proses ekstraksi informasi perusahaan perlu penyesuaian")
            # Pastikan kembali ke halaman results jika ada error
            try:
                if 'current_url' in locals():
                    self.driver.get(current_url)
                    time.sleep(1)
            except:
                pass
            return None
    
    def extract_company_detail_info(self, existing_data=None):
        """
        Ekstrak informasi detail dari halaman detail perusahaan
        Ditingkatkan dengan penanganan kesalahan yang lebih baik dan penggabungan data yang fleksibel
        """
        # Inisialisasi dengan data yang ada jika disediakan, jika tidak gunakan dict kosong
        detail_info = existing_data or {
            'phone': '',
            'email': '',
            'website': '',
            'company_url': '',
            'registered_since': '',
            'logo_url': ''
        }
        
        try:
            # Pastikan kita memiliki URL saat ini sebelum melakukan apa pun
            if not detail_info.get('company_url') or not '/company/' in detail_info['company_url']:
                current_url = self.driver.current_url
                if '/company/' in current_url or '/perusahaan/' in current_url:
                    detail_info['company_url'] = current_url
                    print(f"        ✅ URL perusahaan disimpan: {current_url}")
            
            # Tunggu elemen kunci untuk memastikan halaman telah dimuat
            page_loaded = False
            try:
                # Coba beberapa selector untuk deteksi halaman
                for page_selector in ["div.sisnaker-container", "div.container", "div.text-xs.text-grey-400"]:
                    try:
                        WebDriverWait(self.driver, 3).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, page_selector))
                        )
                        page_loaded = True
                        break
                    except:
                        continue
                
                if not page_loaded:
                    # Jangan tampilkan stacktrace, cukup log tanpa detail error
                    print(f"        ℹ️ Menunggu halaman dimuat secara penuh...")
                    time.sleep(2)  # Berikan waktu tambahan
            except:
                # Hindari stacktrace, lanjutkan saja
                pass
            
            # Ekstrak logo dari halaman detail dengan multiple selectors
            for selector in [
                "div.w-12.h-12.sm\\:w-20.sm\\:h-20 img", 
                "div.w-12.h-12 img",
                "div[class*='h-12'] img",
                "img[class*='rounded']"
            ]:
                try:
                    logo_elem = self.driver.find_element(By.CSS_SELECTOR, selector)
                    logo_url = logo_elem.get_attribute("src")
                    if logo_url and not any(fallback in logo_url.lower() for fallback in ['fallback', 'blank-white']):
                        detail_info['logo_url'] = logo_url
                        print(f"        ✅ Logo URL ditemukan: {logo_url[:30]}...")
                        break
                except:
                    continue
            
            # Ekstrak tanggal registrasi dengan multiple selectors
            for selector in [
                "div.text-xs.sm\\:text-sm.text-grey-400",
                "div.text-xs.text-grey-400",
                "div[class*='text-grey-400']"
            ]:
                try:
                    reg_elem = self.driver.find_element(By.CSS_SELECTOR, selector)
                    reg_text = reg_elem.text.strip()
                    if 'Terdaftar sejak' in reg_text:
                        detail_info['registered_since'] = reg_text.replace('Terdaftar sejak', '').strip()
                        break
                except:
                    continue
            
            # Ekstrak kontak, email, website dari grid dengan robust selector strategy
            grid_selectors = [
                "div.mt-5.sm\\:mt-6.grid div.space-y-1",
                "div.grid div.space-y-1",
                "div[class*='grid'] div[class*='space-y']"
            ]
            
            for grid_selector in grid_selectors:
                try:
                    grid_divs = self.driver.find_elements(By.CSS_SELECTOR, grid_selector)
                    if not grid_divs or len(grid_divs) == 0:
                        continue
                        
                    for grid_div in grid_divs:
                        try:
                            # Try different label and value selectors
                            label_selectors = [
                                "div.text-sm.text-grey-500",
                                "div.text-grey-500",
                                "div[class*='text-grey']"
                            ]
                            value_selectors = [
                                "div.text-sm.font-medium.text-grey-700",
                                "div.font-medium.text-grey-700",
                                "div[class*='font-medium']"
                            ]
                            
                            # Try to find label using multiple selectors
                            label_elem = None
                            for selector in label_selectors:
                                try:
                                    label_elem = grid_div.find_element(By.CSS_SELECTOR, selector)
                                    if label_elem:
                                        break
                                except:
                                    continue
                                    
                            # Try to find value using multiple selectors
                            value_elem = None
                            for selector in value_selectors:
                                try:
                                    value_elem = grid_div.find_element(By.CSS_SELECTOR, selector)
                                    if value_elem:
                                        break
                                except:
                                    continue
                            
                            if label_elem and value_elem:
                                label = label_elem.text.strip().lower()
                                value = value_elem.text.strip()
                                
                                if value:  # Only process if we have a non-empty value
                                    if any(k in label for k in ['kontak', 'phone', 'telepon']):
                                        detail_info['phone'] = value
                                    elif 'email' in label:
                                        detail_info['email'] = value
                                    elif 'website' in label or 'situs' in label:
                                        detail_info['website'] = value
                                        
                        except Exception as field_err:
                            continue
                            
                    # If we found some data with this selector, break the loop
                    if detail_info['phone'] or detail_info['email'] or detail_info['website']:
                        break
                        
                except Exception as grid_err:
                    continue
            
            # Try to find website URL from any anchor tags if not found in grid
            if not detail_info['website']:
                try:
                    # Look for links that might be website URLs
                    links = self.driver.find_elements(By.TAG_NAME, "a")
                    for link in links:
                        href = link.get_attribute("href")
                        text = link.text.strip().lower()
                        if href and ('http://' in href or 'https://' in href) and not 'karirhub' in href:
                            if 'website' in text or 'situs' in text or '.com' in href or '.co.id' in href:
                                detail_info['website'] = href
                                print(f"        ✅ Website URL found from link: {href}")
                                break
                except:
                    pass
            
            print(f"        ✅ Detail diekstrak: phone={detail_info['phone']}, email={detail_info['email']}, website={detail_info['website']}")
            
        except Exception as e:
            print(f"        ⚠️ Error extracting detail info: {str(e)}")
        
        return detail_info
    
    def close(self):
        """Tutup browser"""
        if self.driver:
            self.driver.quit()
            print("🔒 Browser ditutup")
    
    def __del__(self):
        """Destructor untuk memastikan browser ditutup"""
        self.close()

print("✅ KarirHubCrawler dengan Selenium berhasil dibuat!")
print("🔄 Anti-stale element mechanism included - mengatasi masalah stale element reference")
print("📌 Catatan: Chrome driver akan diunduh otomatis jika belum ada")

✅ KarirHubCrawler dengan Selenium berhasil dibuat!
🔄 Anti-stale element mechanism included - mengatasi masalah stale element reference
📌 Catatan: Chrome driver akan diunduh otomatis jika belum ada


## 🎛️ Implementasi Class Widget Interaktif

Cell ini mengimplementasikan class `KarirHubWidget` yang menyediakan antarmuka pengguna interaktif untuk sistem crawling KarirHub™.

### 🏗️ Struktur Class:

#### **Komponen UI:**
- **Input Field**: Untuk memasukkan kata kunci pencarian perusahaan
- **Button**: Tombol untuk memulai proses crawling
- **Output Area**: Menampilkan hasil dan status proses
- **Progress Indicator**: Menunjukkan kemajuan proses scraping

#### **Metode Utama:**
- **`__init__`**: Inisialisasi widget dan layout interface
- **`search_companies`**: Fungsi utama untuk melakukan pencarian dan crawling
- **`display_widget`**: Menampilkan antarmuka widget
- **Event handlers**: Menangani interaksi pengguna dengan widget

### 🔍 Fitur Pencarian:
- Pencarian berdasarkan nama perusahaan
- Filter lokasi dan kategori industri
- Ekstraksi data komprehensif (nama, alamat, deskripsi, kontak)
- Output dalam format yang terstruktur

Class ini mengintegrasikan Selenium WebDriver dengan IPython Widgets untuk memberikan pengalaman pengguna yang intuitif.

In [4]:
# Cell 3: Widget Interface untuk Input dan Control
class KarirHubWidget:
    def __init__(self):
        self.crawler = KarirHubCrawler()
        self.results = []
        self.setup_widgets()
    
    def setup_widgets(self):
        """Setup widget interface with professional responsive design"""
        # Responsive style configuration
        style = {'description_width': '120px'}
        
        # Input widgets with responsive layout - changed to Textarea for multiple keywords
        self.keyword_input = widgets.Textarea(
            value='',
            placeholder='Masukkan keyword perusahaan (satu per baris)\nContoh:\nteknologi\nbank\nstartup',
            description='🔍 Keyword:',
            style=style,
            layout=widgets.Layout(width='100%', max_width='500px', height='100px')
        )
        
        # Control buttons with consistent sizing
        button_layout = widgets.Layout(width='180px', height='45px', margin='5px')
        
        self.search_button = widgets.Button(
            description='🚀 Cari Perusahaan',
            button_style='primary',
            layout=button_layout,
            tooltip='Mulai pencarian otomatis dengan crawling semua halaman'
        )
        
        self.stop_button = widgets.Button(
            description='🛑 Stop Crawling',
            button_style='danger',
            layout=button_layout,
            disabled=True,
            tooltip='Hentikan proses crawling yang sedang berjalan'
        )
        
        # Export buttons with smaller consistent sizing
        export_layout = widgets.Layout(width='140px', height='40px', margin='3px')
        
        self.export_excel_button = widgets.Button(
            description='📊 Excel',
            button_style='success',
            layout=export_layout,
            disabled=True,
            tooltip='Export data ke format Excel (.xlsx)'
        )
        
        self.export_csv_button = widgets.Button(
            description='📋 CSV',
            button_style='success',
            layout=export_layout,
            disabled=True,
            tooltip='Export data ke format CSV'
        )
        
        self.export_json_button = widgets.Button(
            description='📄 JSON',
            button_style='success',
            layout=export_layout,
            disabled=True,
            tooltip='Export data ke format JSON'
        )
        
        self.view_data_button = widgets.Button(
            description='👁️ View Data',
            button_style='info',
            layout=export_layout,
            disabled=True,
            tooltip='Tampilkan data dalam tabel interaktif'
        )
        
        self.clear_button = widgets.Button(
            description='🗑️ Clear All',
            button_style='warning',
            layout=button_layout,
            disabled=True,
            tooltip='Hapus semua data dan reset form'
        )
        
        # Progress bar with full width
        self.progress = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            description='Progress:',
            bar_style='info',
            layout=widgets.Layout(width='100%', max_width='600px', height='25px')
        )
        
        # Status notifikasi dengan animasi
        self.status_label = widgets.HTML(
            value="<style>@keyframes pulse{0%{opacity:1}50%{opacity:0.6}100%{opacity:1}}.status-crawling{background:#059669;color:white;padding:8px 12px;border-radius:6px;text-align:center;font-weight:bold;animation:pulse 1.5s infinite}.status-idle{background:#9ca3af;color:white;padding:8px 12px;border-radius:6px;text-align:center}.status-completed{background:#3b82f6;color:white;padding:8px 12px;border-radius:6px;text-align:center}.status-error{background:#ef4444;color:white;padding:8px 12px;border-radius:6px;text-align:center}</style><div class='status-idle'>🔍 Siap melakukan pencarian</div>",
            layout=widgets.Layout(width='100%', max_width='600px')
        )
        
        # Output area with responsive height
        self.output = widgets.Output(layout=widgets.Layout(
            width='100%',
            max_height='800px',
            overflow='auto'
        ))
        
        # Separate output area for data view
        self.data_view_output = widgets.Output(layout=widgets.Layout(
            width='100%',
            max_height='600px',
            overflow='auto'
        ))
        
        # Bind events
        self.search_button.on_click(self.on_search_clicked)
        self.stop_button.on_click(self.on_stop_clicked)
        self.export_excel_button.on_click(self.on_export_excel_clicked)
        self.export_csv_button.on_click(self.on_export_csv_clicked)
        self.export_json_button.on_click(self.on_export_json_clicked)
        self.view_data_button.on_click(self.on_view_data_clicked)
        self.clear_button.on_click(self.on_clear_clicked)
    
    def display(self):
        """Display the professional widget interface with responsive design"""
        
        # Enhanced header with professional styling
        header = widgets.HTML(value="""
        <div style='
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 25px;
            border-radius: 12px;
            text-align: center;
            margin: 10px 0 20px 0;
            box-shadow: 0 4px 20px rgba(102, 126, 234, 0.3);
            font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif;
        '>
            <h1 style='margin: 0 0 10px 0; font-size: 2.2em; font-weight: 700;'>
                🏢 KarirHub Company Crawler
            </h1>
            <h2 style='margin: 0 0 15px 0; font-size: 1.3em; font-weight: 400; opacity: 0.9;'>
                Auto-Crawl Mode | Professional Data Extraction
            </h2>
            <p style='margin: 0; font-size: 1em; opacity: 0.8; max-width: 600px; margin: 0 auto;'>
                Sistem crawling otomatis dan profesional untuk mengekstrak data perusahaan dari KarirHub dengan tampilan interaktif
            </p>
        </div>
        """)
        
        # Input section with professional styling
        input_section = widgets.VBox([
            widgets.HTML(value="<h3 style='color: #374151; margin: 0 0 15px 0; font-family: \"Segoe UI\", sans-serif;'>📝 Parameter Pencarian</h3>"),
            self.keyword_input,
            widgets.HTML(value="""
            <div style='
                background: #f0f9ff; 
                border: 1px solid #0ea5e9; 
                border-radius: 8px; 
                padding: 12px; 
                margin-top: 15px;
                font-size: 13px;
                color: #0c4a6e;
            '>
                💡 <strong>Tips:</strong> Masukkan satu atau lebih keyword, satu keyword per baris.<br>
                Contoh:<br>
                teknologi<br>
                bank<br>
                startup<br>
                konsultan<br>
                Sistem akan otomatis crawl SEMUA halaman untuk setiap keyword.
            </div>
            """),
        ], layout=widgets.Layout(
            border='1px solid #e5e7eb',
            border_radius='10px',
            padding='20px',
            margin='10px 0',
            background_color='#fafafa'
        ))
        
        # Control section with organized layout
        control_section = widgets.VBox([
            widgets.HTML(value="<h3 style='color: #374151; margin: 0 0 15px 0; font-family: \"Segoe UI\", sans-serif;'>🎛️ Kontrol Sistem</h3>"),
            
            # Main action buttons
            widgets.HTML(value="<h4 style='color: #6b7280; margin: 10px 0 8px 0; font-size: 14px;'>Aksi Utama</h4>"),
            widgets.HBox([
                self.search_button, 
                self.stop_button
            ], layout=widgets.Layout(justify_content='flex-start', flex_wrap='wrap')),
            
            # Progress bar
            widgets.HTML(value="<h4 style='color: #6b7280; margin: 20px 0 8px 0; font-size: 14px;'>Progress Crawling</h4>"),
            self.progress,
            self.status_label,
            
            # Export options
            widgets.HTML(value="<h4 style='color: #6b7280; margin: 20px 0 8px 0; font-size: 14px;'>📤 Opsi Export Data</h4>"),
            widgets.HBox([
                self.export_excel_button,
                self.export_csv_button,
                self.export_json_button
            ], layout=widgets.Layout(justify_content='flex-start', flex_wrap='wrap')),
            
            # View options
            widgets.HTML(value="<h4 style='color: #6b7280; margin: 20px 0 8px 0; font-size: 14px;'>👁️ Tampilan Data</h4>"),
            widgets.HBox([self.view_data_button]),
            self.data_view_output,  # Data view output area
            
            # Management options
            widgets.HTML(value="<h4 style='color: #6b7280; margin: 20px 0 8px 0; font-size: 14px;'>🗂️ Manajemen Data</h4>"),
            widgets.HBox([self.clear_button]),
            
        ], layout=widgets.Layout(
            border='1px solid #e5e7eb',
            border_radius='10px',
            padding='20px',
            margin='10px 0',
            background_color='#fafafa'
        ))
        
        # Features info section
        features_info = widgets.HTML(value="""
        <div style='
            background: linear-gradient(135deg, #10b981 0%, #059669 100%);
            color: white;
            padding: 20px;
            border-radius: 10px;
            margin: 10px 0;
            font-family: "Segoe UI", sans-serif;
        '>
            <h3 style='margin: 0 0 15px 0; font-size: 1.2em;'>⚙️ Fitur Auto-Crawl Mode</h3>
            <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 15px; margin-top: 15px;'>
                <div>
                    <strong>🚀 Crawling Otomatis</strong><br>
                    <small>Mengekstrak SEMUA halaman secara otomatis</small>
                </div>
                <div>
                    <strong>🛑 Kontrol Penuh</strong><br>
                    <small>Tombol stop untuk menghentikan kapan saja</small>
                </div>
                <div>
                    <strong>📊 Data View Interaktif</strong><br>
                    <small>Tabel profesional dengan link yang dapat diklik</small>
                </div>
                <div>
                    <strong>📁 Multi Format Export</strong><br>
                    <small>Excel, CSV, dan JSON dengan timestamp</small>
                </div>
                <div>
                    <strong>🔄 Background Mode</strong><br>
                    <small>Headless Chrome untuk performa optimal</small>
                </div>
                <div>
                    <strong>📱 Responsive Design</strong><br>
                    <small>Layout yang menyesuaikan ukuran layar</small>
                </div>
            </div>
        </div>
        """)
        
        # Footer with trademark
        footer = widgets.HTML(value="""
        <div style='
            background: #f9fafb;
            border: 1px solid #e5e7eb;
            border-radius: 8px;
            padding: 15px;
            text-align: center;
            margin: 20px 0 10px 0;
            font-family: "Segoe UI", sans-serif;
            color: #6b7280;
        '>
            <div style='font-size: 13px; margin-bottom: 8px;'>
                © 2025 <strong style='color: #374151;'>Ferdian Bangkit Wijaya</strong> - UNTIRTA
            </div>
            <div style='font-size: 12px; opacity: 0.8;'>
                KarirHub Company Crawler | Professional Data Extraction Tool
            </div>
        </div>
        """)
        
        # Main layout with responsive container
        main_container = widgets.VBox([
            header,
            input_section,
            control_section,
            features_info,
            self.output,
            footer
        ], layout=widgets.Layout(
            width='100%',
            max_width='1200px',
            margin='0 auto',
            padding='15px'
        ))
        
        # Wrap in responsive container
        responsive_wrapper = widgets.HTML(value="""
        <style>
        .widget-container {
            width: 100%;
            max-width: 1200px;
            margin: 0 auto;
            padding: 10px;
        }
        
        @media (max-width: 768px) {
            .widget-container {
                padding: 5px;
            }
        }
        </style>
        """)
        
        display(responsive_wrapper)
        display(main_container)
    
    def on_search_clicked(self, button):
        """Handle search button click - support multiple keywords separated by newlines"""
        with self.output:
            clear_output(wait=True)
            
            # Validate input
            keyword_text = self.keyword_input.value.strip()
            if not keyword_text:
                print("❌ Mohon masukkan keyword pencarian!")
                return
            
            # Split keywords by newline and filter out empty lines
            keywords = [k.strip() for k in keyword_text.split('\n') if k.strip()]
            
            if not keywords:
                print("❌ Mohon masukkan keyword pencarian yang valid!")
                return
            
            # Disable search button, enable stop button
            self.search_button.disabled = True
            self.stop_button.disabled = False
            self.progress.value = 0
            self.results = []  # Reset results
            
            # We'll use a try-except-finally block to handle errors and ensure buttons are re-enabled
            try:
                print(f"🚀 Memulai auto-crawling untuk {len(keywords)} keyword:")
                for i, kw in enumerate(keywords, 1):
                    print(f"   {i}. {kw}")
                
                print("\n⚙️ Mode: Auto-Crawl dengan Click Pagination")
                print("🔄 Background: Headless Chrome")
                print("🛑 Klik 'Stop Crawling' untuk menghentikan kapan saja")
                
                # Update status label with animated indicator
                self.status_label.value = "<div class='status-crawling'>🔄 SEDANG CRAWLING...</div>"
                
                # Process each keyword
                total_keywords = len(keywords)
                all_results = []
                
                for idx, keyword in enumerate(keywords, 1):
                    # Update progress based on keyword position
                    progress_start = int((idx-1) / total_keywords * 100)
                    progress_end = int(idx / total_keywords * 100)
                    self.progress.value = progress_start
                    
                    print(f"\n📌 [{idx}/{total_keywords}] Processing keyword: '{keyword}'")
                    
                    # Update status label with current keyword
                    self.status_label.value = f"<div class='status-crawling'>🔄 CRAWLING KEYWORD: '{keyword}' ({idx}/{total_keywords})</div>"
                    
                    # Perform search for this keyword
                    keyword_results = self.crawler.search_companies(keyword)
                    
                    # Add keyword identifier to each result
                    for result in keyword_results:
                        result['search_keyword'] = keyword
                    
                    # Add to overall results
                    all_results.extend(keyword_results)
                    
                    # Show interim progress
                    print(f"✅ Keyword '{keyword}': {len(keyword_results)} perusahaan ditemukan")
                    print(f"📊 Total sejauh ini: {len(all_results)} perusahaan")
                    
                    # Update progress
                    self.progress.value = progress_end
                    
                    # Check if user stopped the process
                    if self.crawler.stop_crawling:
                        print(f"\n🛑 Crawling dihentikan pada keyword {idx}/{total_keywords}")
                        self.status_label.value = "<div class='status-error'>🛑 CRAWLING DIHENTIKAN</div>"
                        break
                
                # Store final results
                self.results = all_results
                
                # Update progress
                self.progress.value = 100
                self.status_label.value = "<div class='status-completed'>✅ CRAWLING SELESAI</div>"
                
                # Display results
                self.display_results()
                
                # Enable export buttons if results found
                if self.results:
                    self.export_excel_button.disabled = False
                    self.export_csv_button.disabled = False
                    self.export_json_button.disabled = False
                    self.view_data_button.disabled = False
                    self.clear_button.disabled = False
                    
            except Exception as e:
                print(f"❌ Error during search: {str(e)}")
                # Update status label on error
                self.status_label.value = "<div class='status-error'>❌ CRAWLING GAGAL</div>"
                
            finally:
                # Re-enable search button, disable stop button
                self.search_button.disabled = False
                self.stop_button.disabled = True
    
    def on_stop_clicked(self, button):
        """Handle stop button click"""
        with self.output:
            print("\n🛑 Mengirim signal stop ke crawler...")
            self.crawler.stop()
            self.stop_button.disabled = True
            # Update status label when stopping
            self.status_label.value = "<div class='status-error'>🛑 MENGHENTIKAN CRAWLING...</div>"
            print("⏳ Menunggu halaman saat ini selesai diproses...")
    
    def display_results(self):
        """Display search results summary only"""
        if not self.results:
            print("😔 Tidak ada perusahaan yang ditemukan. Coba keyword lain.")
            return
        
        # Get statistics per keyword
        keyword_stats = {}
        for company in self.results:
            keyword = company.get('search_keyword', 'Unknown')
            if keyword not in keyword_stats:
                keyword_stats[keyword] = 0
            keyword_stats[keyword] += 1
            
        print(f"\n🎉 Berhasil menemukan {len(self.results)} perusahaan!")
        
        # Display keyword statistics
        if keyword_stats:
            print(f"\n📊 Statistik per Keyword:")
            for keyword, count in keyword_stats.items():
                print(f"   🔍 '{keyword}': {count} perusahaan")
                
        print("\n" + "=" * 80)
        
        # Display text summary
        for i, company in enumerate(self.results, 1):
            print(f"\n🏢 {i}. {company['name']}")
            if company.get('search_keyword'):
                print(f"   🔍 Keyword: {company['search_keyword']}")
            if company.get('location'):
                print(f"   📍 Lokasi: {company['location']}")
            if company.get('industry'):
                print(f"   🏭 Industri: {company['industry']}")
            if company.get('jobs_available'):
                print(f"   💼 Lowongan: {company['jobs_available']}")
            if company.get('company_url'):
                print(f"   🌐 URL: {company['company_url']}")
            if company.get('description'):
                print(f"   📄 Deskripsi: {company['description'][:100]}...")
            print("-" * 60)
        
        print(f"\n💡 Klik tombol '👁️ View Data' untuk melihat tabel interaktif dengan fitur scroll dan link yang dapat diklik!")
    
    def on_view_data_clicked(self, button):
        """Handle view data button click"""
        with self.data_view_output:  # Use separate output area
            clear_output(wait=True)
            
            if not self.results:
                print("❌ Tidak ada data untuk ditampilkan!")
                return
            
            print(f"📊 MENAMPILKAN DATA VIEW INTERAKTIF")
            print("=" * 80)
            
            # Display interactive data view
            self.display_data_view()
    
    def display_data_view(self):
        """Display professional interactive data view with DataFrame"""
        if not self.results:
            return
        
        print(f"DATA VIEW - INTERACTIVE TABLE")
        print("=" * 50)
        
        # Create DataFrame
        df = pd.DataFrame(self.results)
        
        # Reorder columns for better display
        display_columns = ['name', 'location', 'industry', 'jobs_available', 'search_keyword', 'phone', 'email', 'website', 'company_url', 'logo_url', 'registered_since', 'description']
        available_columns = [col for col in display_columns if col in df.columns]
        df_display = df[available_columns].copy()
        
        # Rename columns for better display
        column_names = {
            'name': 'Nama Perusahaan',
            'location': 'Lokasi',
            'industry': 'Industri',
            'jobs_available': 'Lowongan',
            'search_keyword': 'Keyword Pencarian',
            'phone': 'Telepon',
            'email': 'Email',
            'website': 'Website',
            'company_url': 'URL Perusahaan',
            'logo_url': 'Logo URL',
            'registered_since': 'Terdaftar Sejak',
            'description': 'Deskripsi'
        }
        df_display = df_display.rename(columns=column_names)
        
        # Create HTML table with enhanced styling and clickable links
        html_content = self.create_professional_table_html(df_display)
        
        # Display the interactive table
        display(HTML(html_content))
        
        print(f"✨ Tabel interaktif menampilkan {len(df_display)} perusahaan")
        print("💡 Tips: Scroll horizontal untuk melihat semua kolom, klik URL untuk membuka halaman perusahaan")
    
    def create_professional_table_html(self, df):
        """Create professional HTML table with enhanced styling and clickable links"""
        
        # Enhanced CSS styling
        css_style = """
        <style>
        .professional-table-container {
            width: 100%;
            max-width: 100vw;
            margin: 20px 0;
            border-radius: 12px;
            box-shadow: 0 4px 20px rgba(0,0,0,0.1);
            overflow: hidden;
            background: white;
            border: 1px solid #e0e6ed;
        }
        
        .table-header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            text-align: center;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        
        .table-header h3 {
            margin: 0;
            font-size: 1.5em;
            font-weight: 600;
        }
        
        .table-header p {
            margin: 5px 0 0 0;
            opacity: 0.9;
            font-size: 0.95em;
        }
        
        .table-scroll-container {
            overflow-x: auto;
            overflow-y: auto;
            max-height: 600px;
            background: white;
        }
        
        .professional-table {
            width: 100%;
            min-width: 1600px;
            border-collapse: separate;
            border-spacing: 0;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            font-size: 14px;
            background: white;
        }
        
        .professional-table th {
            background: linear-gradient(to bottom, #f8fafc, #e2e8f0);
            color: #2d3748;
            font-weight: 600;
            padding: 15px 12px;
            text-align: left;
            border-bottom: 2px solid #cbd5e0;
            position: sticky;
            top: 0;
            z-index: 10;
            white-space: nowrap;
            font-size: 13px;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }
        
        .professional-table td {
            padding: 12px;
            border-bottom: 1px solid #e2e8f0;
            vertical-align: top;
            background: white;
            max-width: 250px;
            word-wrap: break-word;
        }
        
        .professional-table tr:hover td {
            background-color: #f7fafc;
            transition: all 0.2s ease;
        }
        
        .professional-table tr:nth-child(even) td {
            background-color: #f9fafb;
        }
        
        .professional-table tr:nth-child(even):hover td {
            background-color: #f1f5f9;
        }
        
        .company-name {
            font-weight: 600;
            color: #2563eb;
            font-size: 15px;
            max-width: 200px;
        }
        
        .location-cell {
            color: #059669;
            font-weight: 500;
            max-width: 150px;
        }
        
        .industry-cell {
            color: #7c3aed;
            background: #faf5ff;
            padding: 6px 10px;
            border-radius: 6px;
            font-size: 12px;
            font-weight: 500;
            display: inline-block;
            max-width: 180px;
        }
        
        .jobs-cell {
            text-align: center;
            font-weight: 600;
            color: #dc2626;
            background: #fef2f2;
            padding: 6px 12px;
            border-radius: 20px;
            font-size: 13px;
            display: inline-block;
            min-width: 40px;
        }
        
        .url-link {
            color: #2563eb;
            text-decoration: none;
            font-weight: 500;
            padding: 6px 12px;
            background: #eff6ff;
            border: 1px solid #bfdbfe;
            border-radius: 6px;
            display: inline-block;
            transition: all 0.2s ease;
            font-size: 12px;
            max-width: 200px;
            text-overflow: ellipsis;
            overflow: hidden;
            white-space: nowrap;
        }
        
        .url-link:hover {
            background: #1d4ed8;
            color: white;
            text-decoration: none;
            transform: translateY(-1px);
            box-shadow: 0 2px 8px rgba(29, 78, 216, 0.3);
        }
        
        .description-cell {
            max-width: 300px;
            color: #4b5563;
            line-height: 1.4;
            font-size: 13px;
        }
        
        .contact-cell {
            color: #0891b2;
            font-weight: 500;
            max-width: 180px;
        }
        
        .date-cell {
            color: #7c2d12;
            font-size: 12px;
            font-weight: 500;
            max-width: 120px;
        }
        
        .table-footer {
            background: #f8fafc;
            padding: 15px 20px;
            text-align: center;
            border-top: 1px solid #e2e8f0;
            font-size: 12px;
            color: #6b7280;
        }
        
        .trademark {
            margin-top: 10px;
            font-style: italic;
            color: #9ca3af;
        }
        
        .stats-row {
            background: #f0f9ff;
            border: 1px solid #0ea5e9;
        }
        
        .stats-row td {
            background: #f0f9ff !important;
            font-weight: 600;
            color: #0c4a6e;
        }
        
        @media (max-width: 768px) {
            .professional-table {
                min-width: 800px;
                font-size: 12px;
            }
            
            .professional-table th,
            .professional-table td {
                padding: 8px 6px;
            }
        }
        </style>
        """
        
        # Create table header
        table_header = f"""
        <div class="table-header">
            <h3>📊 Data Perusahaan KarirHub</h3>
            <p>Total: {len(df)} perusahaan ditemukan | Scroll horizontal untuk melihat semua data</p>
        </div>
        """
        
        # Create table content
        table_html = "<table class='professional-table'>"
        
        # Table headers
        table_html += "<thead><tr>"
        for col in df.columns:
            table_html += f"<th>{col}</th>"
        table_html += "</tr></thead><tbody>"
        
        # Table rows
        for idx, row in df.iterrows():
            table_html += "<tr>"
            for col in df.columns:
                cell_value = str(row[col]) if pd.notna(row[col]) else ""
                
                if col == "Nama Perusahaan":
                    table_html += f"<td class='company-name'>{cell_value}</td>"
                elif col == "Lokasi":
                    table_html += f"<td class='location-cell'>📍 {cell_value}</td>"
                elif col == "Industri":
                    table_html += f"<td><span class='industry-cell'>🏭 {cell_value}</span></td>"
                elif col == "Lowongan":
                    jobs_text = f"{cell_value} pos" if cell_value and cell_value != "0" else "0"
                    table_html += f"<td><span class='jobs-cell'>💼 {jobs_text}</span></td>"
                elif col == "Telepon":
                    table_html += f"<td class='contact-cell'>📞 {cell_value}</td>"
                elif col == "Email":
                    if cell_value:
                        table_html += f"<td class='contact-cell'>📧 <a href='mailto:{cell_value}' class='url-link'>{cell_value}</a></td>"
                    else:
                        table_html += f"<td class='contact-cell'>-</td>"
                elif col == "Website":
                    if cell_value:
                        # Tambahkan http:// jika tidak ada
                        web_url = cell_value if cell_value.startswith(('http://', 'https://')) else f"http://{cell_value}"
                        table_html += f"<td class='contact-cell'>🌍 <a href='{web_url}' target='_blank' class='url-link'>{cell_value}</a></td>"
                    else:
                        table_html += f"<td class='contact-cell'>-</td>"
                elif col == "URL Perusahaan" and cell_value:
                    # Create clickable link
                    display_url = cell_value.replace("https://karirhub.kemnaker.go.id/perusahaan/", "").replace("http://", "").replace("https://", "")
                    if len(display_url) > 25:
                        display_url = display_url[:25] + "..."
                    table_html += f"<td><a href='{cell_value}' target='_blank' class='url-link'>🌐 {display_url}</a></td>"
                elif col == "Logo URL" and cell_value:
                    if len(cell_value) > 30:
                        display_url = cell_value[:30] + "..."
                    else:
                        display_url = cell_value
                    table_html += f"<td><a href='{cell_value}' target='_blank' class='url-link'>🖼️ {display_url}</a></td>"
                elif col == "Terdaftar Sejak":
                    table_html += f"<td class='date-cell'>📅 {cell_value}</td>"
                elif col == "Keyword Pencarian":
                    # Highlight the search keyword
                    if cell_value:
                        table_html += f"<td><span style='background:#e9f5ff; color:#0369a1; padding:4px 8px; border-radius:4px; font-weight:500;'>🔍 {cell_value}</span></td>"
                    else:
                        table_html += f"<td>-</td>"
                elif col == "Deskripsi":
                    # Truncate description for table display
                    desc = cell_value[:150] + "..." if len(cell_value) > 150 else cell_value
                    table_html += f"<td class='description-cell'>{desc}</td>"
                else:
                    table_html += f"<td>{cell_value}</td>"
            table_html += "</tr>"
        
        table_html += "</tbody></table>"
        
        # Create table footer with statistics
        # Get keyword stats if available
        keyword_stats = ""
        if 'Keyword Pencarian' in df.columns and not df['Keyword Pencarian'].isna().all():
            keyword_counts = df['Keyword Pencarian'].value_counts()
            keyword_stats = "<div style='margin-top:10px; font-size:13px;'>"
            keyword_stats += "<strong>🔍 Statistik per keyword:</strong><br>"
            for kw, count in keyword_counts.items():
                keyword_stats += f"- '{kw}': {count} perusahaan<br>"
            keyword_stats += "</div>"
        
        table_footer = f"""
        <div class="table-footer">
            📈 <strong>Statistik:</strong> {len(df)} perusahaan | 
            🏢 {len(df[df['Industri'].notna()])} dengan info industri | 
            📍 {len(df[df['Lokasi'].notna()])} dengan info lokasi | 
            📞 {len(df[df['Telepon'].notna() & (df['Telepon'] != '')])} dengan info telepon |
            📧 {len(df[df['Email'].notna() & (df['Email'] != '')])} dengan info email |
            🌍 {len(df[df['Website'].notna() & (df['Website'] != '')])} dengan info website |
            💼 {df['Lowongan'].sum()} total lowongan tersedia
            {keyword_stats}
            <div class="trademark">
                © 2025 Ferdian Bangkit Wijaya - UNTIRTA | KarirHub Company Crawler
            </div>
        </div>
        """
        
        # Combine all parts
        complete_html = f"""
        {css_style}
        <div class="professional-table-container">
            {table_header}
            <div class="table-scroll-container">
                {table_html}
            </div>
            {table_footer}
        </div>
        """
        
        return complete_html
    
    def on_export_excel_clicked(self, button):
        """Handle Excel export button click"""
        with self.output:
            if not self.results:
                print("❌ Tidak ada data untuk di-export!")
                return
            
            try:
                # Create DataFrame
                df = pd.DataFrame(self.results)
                
                # Generate filename with timestamp
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"karirhub_companies_{timestamp}.xlsx"
                
                # Export to Excel
                df.to_excel(filename, index=False)
                
                print(f"\n✅ Data berhasil di-export ke Excel: {filename}")
                print(f"📊 Total data: {len(df)} perusahaan")
                print(f"📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                
                # Show data preview
                print(f"\n📋 Preview data (Excel):")
                print(df[['name', 'location', 'industry', 'jobs_available']].head())
                
            except Exception as e:
                print(f"❌ Error saat export Excel: {str(e)}")
    
    def on_export_csv_clicked(self, button):
        """Handle CSV export button click"""
        with self.output:
            if not self.results:
                print("❌ Tidak ada data untuk di-export!")
                return
            
            try:
                # Create DataFrame
                df = pd.DataFrame(self.results)
                
                # Generate filename with timestamp
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"karirhub_companies_{timestamp}.csv"
                
                # Export to CSV
                df.to_csv(filename, index=False, encoding='utf-8')
                
                print(f"\n✅ Data berhasil di-export ke CSV: {filename}")
                print(f"📊 Total data: {len(df)} perusahaan")
                print(f"📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                
                # Show data preview
                print(f"\n📋 Preview data (CSV):")
                print(df[['name', 'location', 'industry', 'jobs_available']].head())
                
            except Exception as e:
                print(f"❌ Error saat export CSV: {str(e)}")
    
    def on_export_json_clicked(self, button):
        """Handle JSON export button click"""
        with self.output:
            if not self.results:
                print("❌ Tidak ada data untuk di-export!")
                return
            
            try:
                # Generate filename with timestamp
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"karirhub_companies_{timestamp}.json"
                
                # Export to JSON
                with open(filename, 'w', encoding='utf-8') as f:
                    json.dump(self.results, f, ensure_ascii=False, indent=2)
                
                print(f"\n✅ Data berhasil di-export ke JSON: {filename}")
                print(f"📊 Total data: {len(self.results)} perusahaan")
                print(f"📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                
                # Show data preview
                print(f"\n📋 Preview data (JSON):")
                if self.results:
                    import json as json_module
                    preview = self.results[:3] if len(self.results) >= 3 else self.results
                    print(json_module.dumps(preview, ensure_ascii=False, indent=2))
                
            except Exception as e:
                print(f"❌ Error saat export JSON: {str(e)}")
    
    def on_clear_clicked(self, button):
        """Handle clear button click - clears everything including keyword"""
        with self.output:
            clear_output(wait=True)
            print("🧹 Clearing all data and inputs...")
            
            # Reset all data
            self.results = []
            
            # Clear keyword input
            self.keyword_input.value = ''
            
            # Reset progress bar
            self.progress.value = 0
            
            # Reset status label
            self.status_label.value = "<div class='status-idle'>🔍 Siap melakukan pencarian</div>"
            
            # Clear data view output completely
            with self.data_view_output:
                clear_output()
            
            # Disable export buttons
            self.export_excel_button.disabled = True
            self.export_csv_button.disabled = True
            self.export_json_button.disabled = True
            self.view_data_button.disabled = True
            self.clear_button.disabled = True
            
            print("✅ All data, results, and keyword cleared!")
            print("🔄 Ready for new auto-crawl search...")
            
            # Reset progress
            self.progress.value = 0
            
            # Reset status label
            self.status_label.value = "<div class='status-idle'>🔍 Siap melakukan pencarian</div>"
            
            print("✅ All data, results, and keyword cleared!")
            print("🔄 Ready for new auto-crawl search...")

print("✅ KarirHubWidget class berhasil dibuat!")

✅ KarirHubWidget class berhasil dibuat!


## 🚀 Inisialisasi dan Aktivasi Widget

Cell ini menginisialisasi instance dari class `KarirHubWidget` yang telah didefinisikan sebelumnya.

### 📋 Proses Inisialisasi:
- **Membuat Instance**: Membuat objek `karirhub_widget` dari class `KarirHubWidget`
- **Setup Interface**: Menyiapkan semua komponen antarmuka pengguna
- **Event Binding**: Menghubungkan event handlers dengan elemen UI
- **Validasi**: Memastikan semua komponen berfungsi dengan baik

### 🎯 Fungsi:
Instance yang dibuat akan menyediakan:
- Antarmuka pencarian yang siap digunakan
- Koneksi ke WebDriver yang telah dikonfigurasi
- Sistem logging dan error handling
- Output formatting yang konsisten

Setelah menjalankan cell ini, widget akan siap untuk digunakan dan dapat dipanggil di cell berikutnya.

In [ ]:
# Cell 4: Inisialisasi dan Menjalankan Widget
# Buat instance widget dan tampilkan
karirhub_widget = KarirHubWidget()

print("🎉 Widget siap digunakan! (Auto-Crawl Mode)")
print("📌 Cara penggunaan:")
print("1. Masukkan satu atau banyak keyword (satu keyword per baris)")
print("2. Klik tombol 'Cari Perusahaan'")
print("3. Sistem akan otomatis crawl SEMUA halaman untuk setiap keyword")
print("4. Hasil akan ditampilkan setelah selesai")
print("5. Export data ke format:")
print("   - 📊 Excel (.xlsx)")
print("   - 📋 CSV (.csv)")
print("   - 📄 JSON (.json)")
print("6. 'Clear All' akan menghapus semua data termasuk keyword")
print("\n⚙️ Auto-Crawl Mode Features:")
print("- Mendukung multiple keyword (satu per baris)")
print("- Crawling SEMUA halaman secara otomatis")
print("- Berhenti ketika menemukan halaman kosong")
print("- Background mode (headless browser)")
print("- Performa optimal dan efisien")
print("- Tidak ada limit halaman")
print("\n" + "="*60)

# Tampilkan widget
karirhub_widget.display()

🔧 Menyiapkan Chrome driver...
✅ Chrome driver siap! (Background mode)
🎉 Widget siap digunakan! (Auto-Crawl Mode)
📌 Cara penggunaan:
1. Masukkan satu atau banyak keyword (satu keyword per baris)
2. Klik tombol 'Cari Perusahaan'
3. Sistem akan otomatis crawl SEMUA halaman untuk setiap keyword
4. Hasil akan ditampilkan setelah selesai
5. Export data ke format:
   - 📊 Excel (.xlsx)
   - 📋 CSV (.csv)
   - 📄 JSON (.json)
6. 'Clear All' akan menghapus semua data termasuk keyword

⚙️ Auto-Crawl Mode Features:
- Mendukung multiple keyword (satu per baris)
- Crawling SEMUA halaman secara otomatis
- Berhenti ketika menemukan halaman kosong
- Background mode (headless browser)
- Performa optimal dan efisien
- Tidak ada limit halaman



HTML(value='\n        <style>\n        .widget-container {\n            width: 100%;\n            max-width: 1…

## 🎮 Eksekusi dan Penggunaan Widget

Cell ini menjalankan dan menampilkan widget interaktif untuk pencarian perusahaan di KarirHub™.

### 📱 Cara Penggunaan:

#### **Step 1: Menjalankan Widget**
- Jalankan cell ini untuk menampilkan antarmuka pencarian
- Widget akan muncul dengan form input dan tombol pencarian

#### **Step 2: Melakukan Pencarian**
1. **Masukkan kata kunci**: Ketik nama perusahaan atau kata kunci terkait
2. **Klik tombol "Cari"**: Mulai proses crawling
3. **Tunggu proses**: Sistem akan mengakses KarirHub dan mengekstrak data
4. **Lihat hasil**: Data perusahaan akan ditampilkan di area output

### 💡 Tips Penggunaan:
- Gunakan kata kunci yang spesifik untuk hasil yang lebih akurat
- Proses crawling membutuhkan waktu tergantung jumlah data yang ditemukan
- Pastikan koneksi internet stabil selama proses berlangsung
- Hasil dapat disimpan atau diproses lebih lanjut sesuai kebutuhan

### ⚠️ Perhatian:
- Gunakan dengan bijak sesuai terms of service KarirHub
- Hindari terlalu banyak request dalam waktu singkat
- Pastikan untuk menutup browser setelah selesai menggunakan

---

## 📊 Hasil dan Analisis Data

Setelah menjalankan sistem crawling KarirHub™, Anda dapat melakukan berbagai analisis terhadap data yang telah dikumpulkan:

### 🔍 Kemungkinan Analisis Lanjutan:
- **Analisis Geografis**: Distribusi perusahaan berdasarkan lokasi
- **Kategorisasi Industri**: Pengelompokan perusahaan berdasarkan sektor
- **Analisis Ukuran**: Klasifikasi berdasarkan skala perusahaan
- **Trend Analysis**: Pola pertumbuhan dan perkembangan perusahaan

### 💾 Export Data:
Data hasil crawling dapat diekspor ke berbagai format:
- **CSV**: Untuk analisis dengan Excel atau tools lainnya
- **JSON**: Untuk integrasi dengan aplikasi web
- **Database**: Untuk penyimpanan jangka panjang
- **Visualization**: Grafik dan chart untuk presentasi

### 🛠️ Pengembangan Lebih Lanjut:
- Implementasi scheduling untuk crawling otomatis
- Integrasi dengan database untuk penyimpanan permanen
- Dashboard analytics untuk visualisasi data
- API endpoint untuk akses data eksternal

---

## 📝 Catatan Penting

### ⚖️ Aspek Legal dan Etika:
- **Robots.txt**: Selalu periksa dan patuhi aturan robots.txt website
- **Rate Limiting**: Gunakan delay yang wajar antar request
- **Terms of Service**: Patuhi syarat dan ketentuan KarirHub
- **Data Privacy**: Hormati privasi data yang dikumpulkan

### 🔧 Maintenance dan Troubleshooting:
- **Update Dependencies**: Perbarui library secara berkala
- **WebDriver Updates**: Pastikan ChromeDriver selalu up-to-date
- **Error Handling**: Monitor dan tangani error yang muncul
- **Performance Monitoring**: Optimasi kinerja sistem crawling

---

## 🏆 Kesimpulan

Sistem crawling KarirHub™ yang telah diimplementasikan menyediakan solusi komprehensif untuk mengumpulkan data perusahaan secara otomatis. Dengan antarmuka yang user-friendly dan arsitektur yang dapat dikembangkan, sistem ini dapat menjadi foundation untuk berbagai aplikasi analisis data perusahaan.

### ✅ Fitur Utama yang Telah Diimplementasikan:
- ✔️ Web scraping otomatis dengan Selenium
- ✔️ Antarmuka interaktif dengan IPython Widgets  
- ✔️ Error handling dan logging yang robust
- ✔️ Output terstruktur dan dapat diolah lebih lanjut
- ✔️ Konfigurasi yang dapat disesuaikan

**Terima kasih telah menggunakan sistem crawling KarirHub™!**

---

**© 2025 - Ferdian Bangkit Wijaya - UNTIRTA**  
*Dikembangkan untuk keperluan penelitian dan analisis data*

**?‍💻 Developer**: Ferdian Bangkit Wijaya  
**🏫 Affiliation**: Universitas Sultan Ageng Tirtayasa (UNTIRTA)  
**📧 Contact**: ferdian.bangkit@untirta.ac.id  
**🔄 Version**: 1.0.0 - Stable Release  
**📅 Last Updated**: Agustus 2025